In [82]:
using HTTP
using JSON
using JSON3
using Dates
using OrderedCollections
include("./ArgoFairEase.jl")
token = ENV["beaconAPItoken"];

In [83]:
datadir = "../data/"
footprintdir = "./Footprint"
mkpath.([datadir, footprintdir])

2-element Vector{String}:
 "../data"
 "./Footprint"

In [77]:
const beacon_services = OrderedDict(
    "Euro-Argo" => "https://beacon-argo.maris.nl",
    "CORA Profile" => "https://databeacon-cora-pr.maris.nl",
    "CORA Timeseries" => "https://beacon-cora-ts.maris.nl",
    "EMODnet Chemistry" => "https://databeacon-emod-chem.maris.nl",
    "World Ocean Database" => "https://beacon-wod.maris.nl",
    "SeaDataNet CDI TS" => "https://beacon-cdi-ts.maris.nl",
    "CMEMS BGC" => "https://beacon-cmems.maris.nl",
);

In [97]:
datasource = "World Ocean Database"
domain = [26.5, 41.95, 40.0, 47.95]
parameter1 = "TEMP"
if datasource == "World Ocean Database"
    parameter1 = "Temperature"
end
parameter2 = "PSAL"
minlon = domain[1]
maxlon = domain[2]
minlat = domain[3]
maxlat = domain[4]
mindepth = 0.
maxdepth = 20.
dateref = Dates.Date(1950, 1, 1) #Days since 1950
mintemporal = (mintemp - dateref).value #Days since 1950
maxtemporal = (maxtemp - dateref).value #Days since 1950
regionname = "BlackSea"
datestart = Dates.Date(2000, 1, 1)
dateend = Dates.Date(2010, 12, 31)
@info(datestart, dateend)

┌ Info: 2000-01-01
└   dateend = 2010-12-31


In [98]:
# datasource = "Euro-Argo"
beaconURL = beacon_services[datasource]
footprintURL = joinpath(beaconURL, "api/datasets/footprint")
datasource_name = replace(datasource, " " => "_")
footprintfile = joinpath(footprintdir, "Footprint_$(datasource_name).json")
@info("Footprint endpoint: $(footprintURL)")
@info("Footprint file: $(footprintfile)")

[ Info: Footprint endpoint: https://beacon-wod.maris.nl/api/datasets/footprint
[ Info: Footprint file: ./Footprint/Footprint_World_Ocean_Database.json


In [87]:
if isfile(footprintfile)
    @info("Footprint file already downloaded")
else
    @info("Writing Footprint file")
    open(footprintfile, "w") do io
        r = HTTP.request("GET", footprintURL, 
            headers = ["Authorization" => "Bearer $(token)"],
            response_stream=io)
        @info(r.status)
    end
end

[ Info: Writing Footprint file
[ Info: 200


In [94]:
# Note 1: the JSON file is 193M, just to get the list of variables?

In [88]:
data = JSON.parsefile(footprintfile);
attributes = data["unique_column_attributes"]
params = sort(collect(keys(attributes)));
"TEMP" in params, "PSAL" in params

(false, false)

In [99]:
parameter1 in params

true

In [100]:
query = ArgoFairEase.prepare_query_new(datasource, parameter1, parameter2, datestart, dateend, 
    mindepth, maxdepth, minlon, maxlon, minlat, maxlat)

"{\"query_parameters\":[{\"column_name\":\"Temperature\",\"alias\":\"Temperature\"},{\"column_name\":\"time\",\"alias\":\"TIME\"},{\"column_name\":\"z\",\"alias\":\"DEPTH\"},{\"column_name\":\"lon\",\"alias\":\"LONGITUDE\"},{\"column_name\":\"lat\",\"alias\":\"LATITUDE\"},{\"column_name\":\"dataset\",\"alias\":\"DATASET" ⋯ 283 bytes ⋯ "e}],\"filters\":[{\"for_query_parameter\":\"TIME\",\"min\":18262,\"max\":22279},{\"for_query_parameter\":\"DEPTH\",\"min\":0.0,\"max\":20.0},{\"for_query_parameter\":\"LONGITUDE\",\"min\":26.5,\"max\":41.95},{\"for_query_parameter\":\"LATITUDE\",\"min\":40.0,\"max\":47.95}],\"output\":{\"format\":\"netcdf\"}}"

In [94]:
# Note 2: cannot work with such a long list of parameters!!

In [101]:
filename = joinpath(datadir, "$(datasource_name)_$(parameter)_$(regionname)_$(Dates.format(datestart, "yyyymmdd"))-$(Dates.format(dateend, "yyyymmdd"))_$(Int(mindepth))-$(Int(maxdepth))m.nc");
filename

"../data/World_Ocean_Database_TEMP_BlackSea_20000101-20101231_0-20m.nc"

In [102]:
@info("Data will be written in file:\n$(filename)")

if isfile(filename)
    @info("File already downloaded")
    rm(filename)
end
begin
    @time open(filename, "w") do io
        r = HTTP.request("POST", joinpath(beaconURL, "api/query"),
            ["Authorization" => "Bearer $(token)", 
                "Content-Type" => "application/json",
            ],
            query,
            response_stream=io);
        @info(r.status)
    end
end;

@info("NetCDF file size: $(round(filesize(filename)/1000^2, digits=1))M")

┌ Info: Data will be written in file:
└ ../data/World_Ocean_Database_TEMP_BlackSea_20000101-20101231_0-20m.nc


  0.683875 seconds (16.76 k allocations: 1.158 MiB, 3.13% compilation time)


[ Info: 204
[ Info: NetCDF file size: 0.0M
